Import all the necessary libraries

In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

Setting the Wiki URL and getting the raw html

In [2]:
# WiKi URL
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Scrape the URL and save the request text
scraped_html = requests.get(url).text

Using BeautifulSoup to create an searchable object and find the table with the Toronto postalcodes

In [3]:
# Create a soup object from the scraped HTML
soup = BeautifulSoup(scraped_html, 'lxml')

# Searching for and setting the postalcode table
TorontoPostalcodes = soup.find('table', class_='wikitable sortable')

After we get the data from WikiPedia we create two dataframes, one for the raw data and one for the cleaned data 

In [4]:
# Creating a dataframe with the raw postalcodes
df = pd.read_html(TorontoPostalcodes.prettify(), flavor=['bs4'], header=0)[0]

# Leaving the unassigned boroughs out
df = df[df.Borough != 'Not assigned']

# Create a dataframe from the raw data and set the index to the Postcode 
df_Postcodes = df.set_index('Postcode')

# Removing duplicate entries from the Postcode dataframe
df_Postcodes = df_Postcodes[~df_Postcodes.index.duplicated(keep='first')]

# Counting the number of occurances for each Postcode and adding it to the raw dataframe
df['freq'] = df.groupby('Postcode')['Postcode'].transform('count')

# Reset the index of the raw dataframe
df = df.reset_index(drop=True)

In [5]:
# How are we doing so far?
df_Postcodes.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M6A,North York,Lawrence Heights
M7A,Queen's Park,Not assigned


Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [6]:
# Loop the raw dataframe and replace the unassigned neighbourhoud to the corresponding borough
for index, row in df[df.Neighbourhood == 'Not assigned'].iterrows():
    df.iloc[index, df.columns.get_loc('Neighbourhood')] = df.iloc[index, df.columns.get_loc('Borough')]

# Loop the raw dataframe and add the neighbourhoods from duplicate postalcodes as a string value to the Postacode dataframe
for index, row in df[df.freq > 1].iterrows():
    df_Postcodes.loc[row.Postcode]['Neighbourhood'] = str(df.loc[df['Postcode'] == row.Postcode]['Neighbourhood'].to_string(header=False, index=False)).replace('\n', ',')


In [7]:
# Reset the index for the Postalcode dataframe
df_Postcodes = df_Postcodes.reset_index()

In [8]:
# Show the cleaned dataframe head
df_Postcodes.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [9]:
# Display the shape of the final dataframe of Toronto's Postcodes from the WikiPedia page
df_Postcodes.shape

(103, 3)

Now Geocode the Postcode dataframe

In [10]:
# First we install the geocode package 
#!conda install -c conda-forge geocoder

In [11]:
# Then we import it
#import geocoder # import geocoder

In [12]:
df_Postcodes.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Not assigned


In [13]:
df_Postcodes = df_Postcodes.set_index('Postcode')

In [14]:
# Add Lat and Lng columns and set to default 0
df_Postcodes['Latitude'] = ''
df_Postcodes['Longitude'] = ''

As stated in the assignment the geocoder is very unreliable; no result came. So using the .csv instead.

In [15]:
# # initialize your variable to None
# lat_lng_coords = None

# # loop until you get the coordinates
# for index, row in df_Postcodes.iterrows():
#     postcode = index
#     print(postcode)
#     while(lat_lng_coords is None):
#         g = geocoder.google('{}, Toronto, Ontario'.format(postcode))
#         lat_lng_coords = g.latlng
#     print(lat_lng_coords)

#     # Add the found coordinates to the Postcode dataframe
#     df_Postcodes[postcode]['latitude'] = lat_lng_coords[0]
#     df_Postcodes[postcode]['longitude'] = lat_lng_coords[1]

In [16]:
csv_geocoder = pd.read_csv('https://cocl.us/Geospatial_data')
csv_geocoder = csv_geocoder.set_index('Postal Code')

In [17]:
for index, row in df_Postcodes.iterrows():
    postcode = index
    df_Postcodes.loc[postcode]['Latitude'] = csv_geocoder.loc[postcode]['Latitude']
    df_Postcodes.loc[postcode]['Longitude'] = csv_geocoder.loc[postcode]['Longitude']
    

In [18]:
df_Postcodes = df_Postcodes.reset_index()
df_Postcodes

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7533,-79.3297
1,M4A,North York,Victoria Village,43.7259,-79.3156
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.6543,-79.3606
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.7185,-79.4648
4,M7A,Queen's Park,Not assigned,43.6623,-79.3895
5,M9A,Etobicoke,Islington Avenue,43.6679,-79.5322
6,M1B,Scarborough,"Rouge,Malvern",43.8067,-79.1944
7,M3B,North York,Don Mills North,43.7459,-79.3522
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.7064,-79.3099
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.6572,-79.3789


Import libraries